In [4]:
import nltk
import random
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm
import json
import pickle
from sklearn.metrics import accuracy_score, f1_score

In [32]:
def load_dataset():  
    dataset=[]
    for root, dirs, files in os.walk('../Dataset'):
        for directory in dirs:
            for root_f, dirs_f, files_f in os.walk(os.path.join(root,directory)):
                files_f.remove('neutral')
                for each_file in files_f:
                    with open(os.path.join(root,directory)+'/'+each_file) as f:
                        data_temp=json.load(f)    
                    if(each_file == 'pos' or each_file == 'sli_pos'):
                        dataset+=[(sentence,1) for sentence in data_temp]
                    elif(each_file == 'neg' or each_file == 'sli_neg'):
                        dataset+=[(sentence,0) for sentence in data_temp]
    return dataset

In [33]:
#execution
labeled_data=load_dataset()    #loading labeled sentences
random.shuffle(labeled_data)

In [34]:
#loading vocab
with open('../NaiveBayes/vocab') as f:
    vocab=json.load(f)

In [35]:
len(vocab)

307

In [42]:
print labeled_data[45]

(u'provides great environment grow', 1)


In [43]:
print len(labeled_data)

118577


In [55]:
vectorizer = CountVectorizer(vocabulary=vocab,min_df=1)
corpus=[sentence for sentence,pol in labeled_data]
features=vectorizer.fit_transform(corpus[:-50000])
labels=[label for sentence,label in labeled_data[:-50000]]

In [56]:
classifier=svm.SVC(kernel='poly',probability=True)

In [57]:
classifier.fit(features,labels)
with open('svc_unigram_model','w') as f:
    pickle.dump(classifier,f) 

In [58]:
#accuracy
with open('svc_unigram_model') as f:
    classifier=pickle.load(f)
features_test=vectorizer.fit_transform(corpus[-10000:])
predicted = classifier.predict(features_test)
accuracy=accuracy_score(labels[-10000:],predicted)
with open('accuracy_svc_unigram','w') as f:                #compute accuracy and store it
    json.dump(accuracy,f) 

In [64]:
print 'Accuracy : '+ str(accuracy)    # accuracy for  linear = 79.65%

Accuracy : 0.8456


In [63]:
#testing
feature_test=[]
print 'sample :'+corpus[544]
# creating feature set
for word in vocab: 
    if word in corpus[50005]:
        feature_test.append(1)
    else:
        feature_test.append(0)
#predicting class        
classifier.predict(feature_test)

sample :job variety awesome


/home/pranav/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([1])